In [41]:
pip install pyarrow==7.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [42]:
# Dataset - https://github.com/bigdata-vandy/spark-taxi/blob/master/yellow_tripdata_2015-01.csv

import pandas as pd

df = pd.read_csv('yellow_tripdata_2015-01.csv')

In [43]:
from pyarrow import csv
%timeit
arrowtable = csv.read_csv("yellow_tripdata_2015-01.csv")

In [44]:
df2 = arrowtable.to_pandas()

In [45]:
df['passenger_count'].sum() == df2['passenger_count'].sum()

True

-----------------------------

In [46]:
import pandas as pd
import pyarrow.parquet as pq

df_p = pd.read_parquet("yellow_tripdata_2022-01.parquet")
arrow_table = pq.read_table("yellow_tripdata_2022-01.parquet")

Using Memory Mapped Files using Apache Arrow

In [47]:
import pyarrow as pa

import os
import psutil
memory_init = psutil.Process(os.getpid()).memory_info().rss >> 20
mmap = pa.memory_map("yellow_tripdata_2015-01.csv", "r") # Which means no memory is allocated
memory_post_csv = psutil.Process(os.getpid()).memory_info().rss >> 20
print(f"csv memory consumption: {memory_post_csv - memory_init}\n")
#print("Memory Consumption: {} MB".format(pa.total_allocated_bytes() >> 20))

csv memory consumption: 0



This memory-mapped file can be read in batches, so that we don't need to load all the contents of the file in memory

In [48]:
from pyarrow.csv import open_csv

# Create a CSVStreamingReader from the memory-mapped file
reader = open_csv(mmap)

# Iterate over all the batches of the file
reader.read_next_batch().to_pandas()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,N,-73.974785,40.750618,1,12.0,1.0,0.5,3.25,0.0,0.3,17.05
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,1,N,-73.994415,40.759109,1,14.5,0.5,0.5,2.00,0.0,0.3,17.80
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,N,-73.951820,40.824413,2,9.5,0.5,0.5,0.00,0.0,0.3,10.80
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,1,N,-74.004326,40.719986,2,3.5,0.5,0.5,0.00,0.0,0.3,4.80
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,1,N,-74.004181,40.742653,2,15.0,0.5,0.5,0.00,0.0,0.3,16.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,1,2015-01-07 20:40:04,2015-01-07 20:42:07,1,0.30,0.000000,0.000000,1,N,0.000000,0.000000,1,3.5,0.5,0.5,0.95,0.0,0.3,5.75
995,1,2015-01-07 20:40:04,2015-01-07 20:43:01,1,0.50,-73.993576,40.741859,1,N,-73.993057,40.745491,1,4.0,0.5,0.5,1.59,0.0,0.3,6.89
996,1,2015-01-07 20:40:04,2015-01-07 21:02:43,2,4.20,-74.001778,40.739468,1,N,-73.953728,40.767357,1,17.5,0.5,0.5,2.00,0.0,0.3,20.80
997,1,2015-01-07 20:40:04,2015-01-07 20:47:36,2,1.20,-74.002380,40.737835,1,N,-73.985878,40.727940,2,7.0,0.5,0.5,0.00,0.0,0.3,8.30
